In [1]:
import numpy as np
import random
from random import shuffle
import os
import gensim
import re
import pickle
import tensorflow as tf
import operator
import math
import sys
from copy import deepcopy
from collections import Counter
from tensorflow.python.ops import rnn, rnn_cell
from tensorflow.python.framework import ops
from tensorflow.models.rnn.translate import seq2seq_model
import string
from pyspark import SparkContext, SparkConf
from pyspark.sql import SQLContext
from pyspark.sql.functions import split, explode
from pyspark.sql import functions as func
from pyspark.sql.types import *
import re

In [2]:
sc = SparkContext('local[4]')
sqlcontext = SQLContext(sc)

In [3]:
def replace_punct(line):    
    punctuation = [x for x in list(string.punctuation)]
    space_punct = [' {0}'.format(elem) for elem in punctuation]
    replace_punctuation = str.maketrans(dict(zip(punctuation, space_punct)))
    line = line.translate(replace_punctuation)
    return line.lower()

In [4]:
def preproccessData(filename):
    
    data_dir = '/media/ai2-rey/data_disk/data_sets/SQuAD/'
    df = sqlcontext.read.json(data_dir+filename)
    df = df.withColumn('data', explode('data'))
    df = df.select('data.paragraphs')
    df = df.withColumn('paragraphs', explode('paragraphs'))
    df = df.select(['paragraphs.context','paragraphs.qas'])
    df = df.withColumn('qas', explode('qas'))
    df = df.selectExpr(['context as context','qas.question as question','qas.answers.text as answer'])
    df = df.withColumn('answer', explode('answer'))
    
    lowercase_UDF = func.UserDefinedFunction(replace_punct, StringType())

    df = (df.withColumn('context_lower', lowercase_UDF(df.context))
            .withColumn('question_lower', lowercase_UDF(df.question))
            .withColumn('answer_lower', lowercase_UDF(df.answer))
            .drop('context')
            .drop('question')
            .drop('answer')
            .withColumnRenamed('context_lower', 'context')
            .withColumnRenamed('question_lower', 'question')
            .withColumnRenamed('answer_lower', 'answer'))
    
    print(df.printSchema())
    pandas_df = df.toPandas()
    context = df.select("context").rdd.flatMap(lambda x: x).collect()
    question = df.select("question").rdd.flatMap(lambda x: x).collect()
    answer = df.select("answer").rdd.flatMap(lambda x: x).collect()
    
    return pandas_df, context, question, answer

In [5]:
train_pandas, train_context, train_question, train_answer = preproccessData('train-v1.1.json')

root
 |-- context: string (nullable = true)
 |-- question: string (nullable = true)
 |-- answer: string (nullable = true)

None


NameError: name 'trian_pandas' is not defined

In [6]:
train_pandas.head()

,context,question,answer
0,"architecturally , the school has a catholic ch...",to whom did the virgin mary allegedly appear i...,saint bernadette soubirous
1,"architecturally , the school has a catholic ch...",what is in front of the notre dame main buildi...,a copper statue of christ
2,"architecturally , the school has a catholic ch...",the basilica of the sacred heart at notre dame...,the main building
3,"architecturally , the school has a catholic ch...",what is the grotto at notre dame ?,a marian place of prayer and reflection
4,"architecturally , the school has a catholic ch...",what sits on top of the main building at notre...,a golden statue of the virgin mary


In [ ]:
test_context, test_question, test_answer = preproccessData('dev-v1.1.json')

In [ ]:
data_dir = '/media/ai2-rey/data_disk/data_sets/SQuAD/'
df_train = sqlcontext.read.json(data_dir+'train-v1.1.json')
df_test = sqlcontext.read.json(data_dir+'dev-v1.1.json')

In [ ]:
df_train = df.withColumn('data', explode('data'))
df_test = df.withColumn('data', explode('data'))

In [ ]:
df.columns

In [ ]:
df.count()

In [ ]:
df = df.select('data.paragraphs')
df = df.withColumn('paragraphs', explode('paragraphs'))
df.printSchema()

In [ ]:
df.count()

In [ ]:
df = df.select(['paragraphs.context','paragraphs.qas'])

In [ ]:
df.printSchema()

In [ ]:
df.count()

In [ ]:
df = df.withColumn('qas', explode('qas'))
df.printSchema()

In [ ]:
df.count()

In [ ]:
df = df.selectExpr(['context as context','qas.question as question','qas.answers.text as answer'])
df = df.withColumn('answer', explode('answer'))
df.printSchema()

In [ ]:
df.take(7)[6]

In [ ]:
def replace_punct(line):    
    punctuation = [x for x in list(string.punctuation)]
    space_punct = [' {0}'.format(elem) for elem in punctuation]
    replace_punctuation = str.maketrans(dict(zip(punctuation, space_punct)))
    line = line.translate(replace_punctuation)
    return line.lower()

lowercase_UDF = func.UserDefinedFunction(replace_punct, StringType())

df = (df.withColumn('context_lower', lowercase_UDF(df.context))
                .withColumn('question_lower', lowercase_UDF(df.question))
                .withColumn('answer_lower', lowercase_UDF(df.answer))
                .drop('context')
                .drop('question')
                .drop('answer')
                .withColumnRenamed('context_lower', 'context')
                .withColumnRenamed('question_lower', 'question')
                .withColumnRenamed('answer_lower', 'answer'))

In [ ]:
df.take(1)

In [ ]:
pandas_df = df.toPandas()

In [ ]:
pandas_df.head()

In [ ]:
context = df.select("context").rdd.flatMap(lambda x: x).collect()
question = df.select("question").rdd.flatMap(lambda x: x).collect()
answer = df.select("answer").rdd.flatMap(lambda x: x).collect()

In [ ]:
word_counter = {}

def count_words(text):
    for line in text:
        for word in line.split():
            word_counter[word] = word_counter.get(word, 0) + 1

count_words(context)

In [ ]:
word_counter['school']

In [ ]:
class QA_Dataset:
    """ This class parses and preprocesses context-utterance datasets for Information Retrieval models
    
    Attributes: 
        train_file (str)        : directory location where the train dataset is located
        val_file (str)          : directory location where the validation dataset is located
        test_file (str)         : directory location where the test dataset is located
        model_type (str)        : type of model that data will be used for (e.g. TF-IDF, LSTM, etc.) 
        context_pad_len (int)   : length that context sequences will be truncated to
        utterance_pad_len (int) : length that utterance sequences will be truncated to
        max_vocab_size (int)    : number of words that the model will recognize
        max_data_size (int)     : number of data samples to load
        word_counter (dict)     : dictionary to count the instances of each word in the train data
        vocab (set)             : set of unique words in the train data
        train_text_data (list)  : list containing all samples of training data in text sequence form. 
        val_text_data (list)    : list containing all samples of validation data in text sequence form
        test_text_data (list)   : list containing all samples of test data in text sequence form
        num_tokens (int)        : number of unique tokens that will be embedded
        word2id_dict (dict)     : dictionary that converts words into unique ids
        id2word_dict (dict)     : dictionary that converts ids into unique words
        train_id_data (list)    : list containing all samples of training data in token sequence form
        val_id_data (list)      : list containing all samples of validation data in token sequence form
        test_id_data (list)     : list containing al samples of test data in token sequence form
            
    """  
    def __init__(self, train_file, val_file, test_file, model_type, context_pad_len=None, utterance_pad_len=None, 
                 max_vocab_size=None, max_data_size=None):
        """ Load data and apply preprocessing based on model type
        
        Args: 
            train_file (str)        : directory location where the train dataset is located
            val_file (str)          : directory location where the validation dataset is located
            test_file (str)         : directory location where the test dataset is located
            model_type (str)        : type of model that data will be used for (e.g. TF-IDF, LSTM, etc.) 
            context_pad_len (int)   : length that context sequences will be truncated to
            utterance_pad_len (int) : length that utterance sequences will be truncated to
            max_vocab_size (int)    : number of words that the model will recognize
            max_data_size (int)     : number of data samples to load
            
        """
        assert model_type in {'dual_LSTM'}
        
        self.train_file = train_file
        self.test_file = test_file
        self.model_type = model_type
        self.context_pad_len = context_pad_len
        self.utterance_pad_len = utterance_pad_len
        self.max_vocab_size = max_vocab_size
        self.max_data_size = max_data_size
        self.word_counter = dict()
        self.vocab = set()
        self.train_text_data = None
        self.val_text_data = None
        self.test_text_data = None
        self.num_tokens = None 
        
        self.__load_data()
        self.word2id_dict, self.id2word_dict = self.__create_word2id_dict()
        self.train_id_data = self.__tokenize_sentences(self.train_text_data, 'train')
        self.val_id_data = self.__tokenize_sentences(self.val_text_data, 'val')
        self.test_id_data = self.__tokenize_sentences(self.test_text_data, 'test')
        
        if self.context_pad_len == None:
            self.context_pad_len, min_context_len, avg_context_len = self.__get_max_sequence_length(zip(*self.train_id_data)[0])
            print 'Context Lengths: max = {}, min = {}, avg = {}'.format(self.context_pad_len, min_context_len, avg_context_len)
        if self.utterance_pad_len == None:
            self.utterance_pad_len, min_utterance_len, avg_utterance_len = self.__get_max_sequence_length(zip(*self.train_id_data)[1])
            print 'Utterance Lengths: max = {}, min = {}, avg = {}'.format(self.utterance_pad_len, min_utterance_len, avg_utterance_len)
            
        self.train_id_data = self.__pad_sequences(self.train_id_data, 'train')
        self.val_id_data = self.__pad_sequences(self.val_id_data, 'val')
        self.test_id_data = self.__pad_sequences(self.test_id_data, 'test')
        
    def __update_word_counter(self, sequence):
        """ Update word_counter with counts for words in a sentence
        
        Args:
            sequence (list<str>) : list of words in a sequence
        
        """
        for word in sequence:
            self.word_counter[word] = self.word_counter.get(word, 0) + 1
            
    def __create_vocab(self):
        """ Create set of most frequent unique words found in the training data """
        
        if self.max_vocab_size == None:
            self.vocab == set(self.word_counter.keys())
        else:
            self.vocab = set(sorted(self.word_counter, key=self.word_counter.get, reverse=True)[:self.max_vocab_size])
        
    def __parse_data(self, df, phase):
        """ Convert raw data into sets of word sequences
        
        Args:
            df (pandas.dataframe) : dataframe containing raw text samples loaded from file
            phase (str)           : defines format in which to parse the data (train, val, test)
            
        Returns:
            list<tuple> : returns a list of tuples where each tuple is a sample of context sequences,
                          utterance sequences, and labels (for train phase) or distractor sequences (for val and test phases)
                          
        """
        assert phase in {'train', 'val', 'test'} 
        parsed_text_data = []
        
        if phase == 'train':
            for index, row in df.iterrows():
                context = row['Context'].split(' ')
                utterance = row['Utterance'].split(' ')
                label = row['Label']
                
                self.__update_word_counter(context)
                self.__update_word_counter(utterance)
                
                parsed_text_data.append((context, utterance, label))
            self.__create_vocab()
        elif phase == 'test' or phase == 'val':
            for index, row in df.iterrows():
                context = row['Context'].split(' ')
                gtu = row['Ground Truth Utterance'].split(' ')
                num_distractors = len(row) - 2
                distractors_list = []
                for i in range(num_distractors):
                    d_i = row[i+2].split(' ')
                    distractors_list.append(d_i)
                
                parsed_text_data.append((context, gtu) + tuple(distractors_list))
        return parsed_text_data
                
    def __load_data(self):
        """ Loads raw train, validation, and test data from file into pandas dataframes """
        
        train_df = pd.read_csv(self.train_file)
        val_df = pd.read_csv(self.val_file)
        test_df = pd.read_csv(self.test_file)
        
        if self.max_data_size != None:
            train_df = train_df.sample(self.max_data_size)

        self.train_text_data = self.__parse_data(train_df, 'train')
        self.val_text_data = self.__parse_data(val_df, 'val')
        self.test_text_data = self.__parse_data(test_df, 'test')
        
        random.shuffle(self.train_text_data)
        random.shuffle(self.val_text_data)
        random.shuffle(self.test_text_data)
        
        del train_df, val_df, test_df
        
    def __create_word2id_dict(self):
        """ Create a dictionary to map each word in the vocab set to a unique id 
        
        Returns:
            dict, dict : A dictionary mapping words to ids and a dictionary mapping ids to words
            
        """
        word2id_dict = dict()
        
        if self.model_type == 'dual_LSTM':
            word2id_dict['PAD'] = 0
            word2id_dict['UNK'] = 1
            
        for word in self.vocab:
            word2id_dict[word] = len(word2id_dict)
            
        self.vocab.add('PAD')
        self.vocab.add('UNK')
        id2word_dict = dict(zip(word2id_dict.values(), word2id_dict.keys()))
        self.num_tokens = len(word2id_dict)
        return word2id_dict, id2word_dict
    
    def __convert_seq2id(self, sequence):
        """ Convert a sequence of text into a sequence of ids 
        
        Returns:
            list<int> : a list of integers where each integer is a word id
            
        """
        sequence_ids = []
        for word in sequence:
            try:
                word_id = self.word2id_dict[word]
            except:
                word_id = self.word2id_dict['UNK']
            sequence_ids.append(word_id)
        return sequence_ids
    
    def __tokenize_sentences(self, text_data, phase):
        """ Convert text data into tokenized id data 
        
        Args:
            text_data (list<tuple>) : list of sample tuples with text sequence data
            phase (str)             : defines format in which to process the data (train, val, test)
            
        Returns:
            list<tuple> : list of sample tuples with tokenized id sequence data
        
        """
        assert phase in {'train', 'val', 'test'} 
        id_data = []
        
        if phase == 'train':
            for sample in text_data:
                context_ids, utterance_ids = (self.__convert_seq2id(seq) for seq in sample[:2])
                id_data.append((context_ids, utterance_ids, sample[2]))
        elif phase == 'val' or phase == 'test':
            for sample in text_data:
                id_data.append(tuple([self.__convert_seq2id(seq) for seq in sample]))
        return id_data
    
    def __get_max_sequence_length(self, sequences):
        """ Finds the maximum, minimum, and average length of a set of sequences 
        
        Args:
            sequences (list<list>) : list of sequences 
        
        Returns:
            int : max sequence length
            int : minimum sequence length
            int : average sequence length
            
        """
        max_len = 0
        min_len = 10000
        avg_len = 0
        for sequence in sequences:
            max_len = max(max_len, len(sequence))
            min_len = min(min_len, len(sequence))
            avg_len += len(sequence)
        avg_len = int(float(avg_len) / len(sequences))
        return max_len, min_len, avg_len
    
    def __apply_padding(self, sequence, pad_len):
        """ Pads a set of sequences to equal length
        
        Args:
            sequence (list<int>) : list of id tokens
            pad_len (int)        : desired length to pad/truncate all sequences to
        
        Returns:
            list<int> : list of id tokens representing padded sequence
        
        """
        padded_sequence = sequence[:]
        
        if len(sequence) < pad_len:
            for i in range(pad_len - len(sequence)):
                padded_sequence.append(0)
        elif len(sequence) > pad_len:
            padded_sequence = sequence[:pad_len]
        return padded_sequence
    
    def __pad_sequences(self, id_data, phase):
        """ Adds padding to all data and pairs each sequence to its original length 
        
        Args: 
            id_data (list<tuple>) : list of sample tuples with id sequence data
            phase (str)           : defines format in which to process the data (train, val, test)
            
        Returns:
            list<tuple> : list of sample tuples with padded id sequences pair with original lengths
            
        """
        assert phase in {'train', 'val', 'test'} 
        
        if self.model_type == 'dual_LSTM':
            padded_id_data = []
            
            if phase == 'train':
                for sample in id_data:
                    padded_id_data.append(((self.__apply_padding(sample[0], self.context_pad_len), 
                                           min(len(sample[0]), self.context_pad_len)),
                                          (self.__apply_padding(sample[1], self.utterance_pad_len), 
                                           min(len(sample[1]), self.utterance_pad_len)),
                                          sample[2]))
            elif phase == 'val' or phase =='test':
                for sample in id_data:
                    padded_id_data.append(((self.__apply_padding(sample[0], self.context_pad_len), 
                                           min(len(sample[0]), self.context_pad_len)),) + 
                                          tuple([(self.__apply_padding(seq, self.utterance_pad_len), 
                                            min(len(seq), self.utterance_pad_len)) for seq in sample[1:]]))
            return padded_id_data